In [3]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten, BatchNormalization
from keras import backend as K


# DATASET_PREFIX = 'drive/MyDrive/idl/datasets/'

DATASET_PREFIX = '/kaggle/input/clock-images/'
labels = np.load(DATASET_PREFIX + 'labels.npy')
images = np.load(DATASET_PREFIX + 'images.npy')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [47]:
class clock_CNN:
    def __init__(self, images, labels,
                num_classes=12,
                batch_size=128,
                num_epochs=20,
                ):
        self.num_classes = num_classes
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        self.image_shape = (150, 150, 1)
        self.filename = 'test'

        self.prep_data(images, labels)
        self.build_model()


    def build_model(self):
        self.model = keras.models.Sequential([        
            Conv2D(64, kernel_size=15, strides=4, activation='relu', input_shape=self.image_shape),
            MaxPooling2D(pool_size=4, strides=4),
            BatchNormalization(),
#             Conv2D(64, kernel_size=3, strides=1, activation='relu'),
#             MaxPooling2D(pool_size=2, strides=2),
#             BatchNormalization(),
            Conv2D(16, kernel_size=3, strides=1, activation='relu'),
            MaxPooling2D(pool_size=2, strides=2),
            BatchNormalization(),
            Flatten(),
            Dropout(0.2),
            Dense(64, activation='relu'),
            Dropout(0.2),
            Dense(128, activation='relu'),
            Dense(self.num_classes, activation='softmax'),
        ])

        self.model.summary()
        
    def convert_labels(self,labels):
        """
        Converts tuple labels to integers
        """
        return tf.math.floor((labels[:,0] + labels[:,1]/60)/12 * self.num_classes)
        # return 2*labels[:,0] + labels[:,1]//30

    def prep_data(self, X_full, y_full):
        validation_fraction = 0.1
        test_fraction = 0.15
        
        num_validation = int(validation_fraction * len(y_full))
        num_testing = int(test_fraction * len(y_full))
        num_train = len(y_full) - num_validation - num_testing
        
        rng = np.random.default_rng(seed=10)
        shuffled_indices = rng.permutation(len(labels))
        y_full = y_full[shuffled_indices]
        X_full = X_full[shuffled_indices]

        y_full = self.convert_labels(y_full)
        y_full = keras.utils.to_categorical(y_full, num_classes = self.num_classes)

        X_full = X_full.reshape(len(X_full),*self.image_shape)/255

        X_train, X_valid, X_test = X_full[:num_train], X_full[num_train:num_train+num_validation], X_full[num_train+num_validation:]
        y_train, y_valid, y_test = y_full[:num_train], y_full[num_train:num_train+num_validation], y_full[num_train+num_validation:]

        self.train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(self.batch_size)
        self.test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(self.batch_size)
        self.valid_dataset = tf.data.Dataset.from_tensor_slices((X_valid, y_valid)).batch(self.batch_size)

    def cyclic_loss(self, y_true, y_pred):
        linear_errors = tf.math.abs(y_true - y_pred)
        errors = tf.minimum(self.num_classes - linear_errors, linear_errors)
        # mean_error = tf.reduce_mean(tf.math.pow(errors,3), axis=0)
        mean_error = tf.reduce_mean(errors, axis=0)
        return mean_error

    def mean_deviation_minutes(self, y_true, y_pred):
        "returns mean error in minutes"
        y_pred = tf.cast(tf.math.argmax(y_pred, axis=-1), tf.float32)
        y_true = tf.cast(tf.math.argmax(y_true, axis=-1), tf.float32)

        linear_errors = tf.math.abs(y_true - y_pred)
        errors = tf.minimum(self.num_classes - linear_errors, linear_errors)
        
        mean_error = tf.reduce_mean(errors, axis=0) / self.num_classes * 720
        return mean_error
    
    def train_model(self):
        self.model.compile(
                loss='categorical_crossentropy',
                optimizer=keras.optimizers.Nadam(learning_rate=.001, beta_1=0.9, beta_2=0.999),
                metrics=['accuracy', self.mean_deviation_minutes],
        )
        self.history = self.model.fit(self.train_dataset,
                                      epochs=self.num_epochs,
                                      validation_data=self.valid_dataset,
                                      batch_size=self.batch_size,
                                      )


In [48]:
myCNN = clock_CNN(images, labels, num_classes=12, batch_size=128, num_epochs=20)
myCNN.train_model()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 34, 34, 64)        14464     
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 8, 8, 64)         0         
 g2D)                                                            
                                                                 
 batch_normalization_10 (Bat  (None, 8, 8, 64)         256       
 chNormalization)                                                
                                                                 
 conv2d_11 (Conv2D)          (None, 6, 6, 16)          9232      
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 3, 3, 16)         0         
 g2D)                                                            
                                                      

In [51]:
myCNN.model.evaluate(myCNN.test_dataset)

22/22 [==============================] - 1s 13ms/step - loss: 1.2011 - accuracy: 0.5163 - common_sense_accuracy: 37.3580


[1.2011046409606934, 0.5162962675094604, 37.35795593261719]

In [49]:
subset = myCNN.test_dataset.take(1)
output = myCNN.model.predict(subset)
y_pred = tf.math.argmax(output, axis=-1)

truelabels = list(subset.as_numpy_iterator())[0][1]
y_true = tf.math.argmax(truelabels, axis=-1)

print(tf.math.abs(y_true - y_pred))

1/1 [==============================] - 1s 536ms/step
tf.Tensor(
[ 0  0  1  1  0  7  0  1  1  0  1  1  0  1  0  0  0  1 11  0  2  8  1  0
  0  1  0  0 11 11  1  0  0  1  0  1  0  1  1  1  1  1  1  1  1  0  1  1
  0  0  1  1  1  0  0  1  0  0  0  0  9  0  1  1  0  1  1  1  1  5  6  0
  1  0  1  1  1 11 11  1  1  1  0  9  0  0  0  0  0 11  0  0  0  0  0  1
  0 11  0  1  0  1  0  0  1  0  1  0 11  0  0  1  1  2  1  1  1  0  1 11
  0  1  0  0  0  1  1  0], shape=(128,), dtype=int64)


In [50]:
def common_sense_accuracy(y_true, y_pred):
    "returns mean error in minutes"
    num_classes = 12
    print(y_pred)
    print(y_true)
    y_pred = tf.cast(y_pred, tf.float32)
    y_true = tf.cast(y_true, tf.float32)
    linear_errors = tf.math.abs(y_true - y_pred)
    print(linear_errors)
    errors = tf.minimum(num_classes - linear_errors, linear_errors)
    print(errors)
    mean_error = tf.reduce_mean(errors, axis=0)/num_classes*720
    return mean_error

print(common_sense_accuracy(y_true, y_pred))

tf.Tensor(
[ 1  1  2 10 11 11  1  2  2  6  8  3  0  2  9  5  9  2  0  6  9 10  0 11
  2  1  0  8  0  0  4  7  0  4  1  1  0  7 11  4  8  9  9 11  7  2  3  3
  0 11 10  5  6  1  5  3  4 11  9  5  9  5  4  5 11 10  5  5  3  7 10  7
  2  4  6  3  3  0  0  2  4  8  6  9  0  9 11  7  4  0  1  3  0  1 11  1
  7 11 11  8  7  6 11  2  5  7  8 10 11  1  3  2  8  3  9  2  9  0  7 11
  2 11  0  8  3  4 11  1], shape=(128,), dtype=int64)
tf.Tensor(
[ 1  1  1  9 11  4  1  3  3  6  9  2  0  1  9  5  9  3 11  6 11  2  1 11
  2  2  0  8 11 11  3  7  0  3  1  2  0  8 10  3  7  8 10 10  8  2  2  2
  0 11  9  6  7  1  5  4  4 11  9  5  0  5  3  6 11 11  6  6  4  2  4  7
  3  4  7  4  4 11 11  1  3  7  6  0  0  9 11  7  4 11  1  3  0  1 11  2
  7  0 11  9  7  5 11  2  4  7  9 10  0  1  3  1  9  5  8  1  8  0  6  0
  2 10  0  8  3  3 10  1], shape=(128,), dtype=int64)
tf.Tensor(
[ 0.  0.  1.  1.  0.  7.  0.  1.  1.  0.  1.  1.  0.  1.  0.  0.  0.  1.
 11.  0.  2.  8.  1.  0.  0.  1.  0.  0. 11. 11.  1.  0.